In [83]:
import os

In [84]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import cv2 as cv
from tqdm import tqdm
import numpy as np

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        aux = os.path.join(dirname, filename)
        if ".jpg" in aux or ".png" in aux:
                files.append(aux)

In [85]:
len(files)

In [86]:
X = []
y = []

for file in tqdm(files):
    img = cv.imread(file)
    img = cv.resize(img, (50,50))
    
    X.append(img)
    y.append(file.split("/")[-2])

In [87]:
X = np.array(X)
y = np.array(y)

In [88]:
for unique in np.unique(y):
    if "GT" in unique:
        y = np.where(y==unique,unique[:-3],y)

In [89]:
np.unique(y)

In [90]:
y = np.where(y=="Gilt-Head Bream","Gilt Head Bream",y)
y = np.where(y=="Horse Mackerel","Hourse Mackerel",y)

In [91]:
np.unique(y)

In [92]:
num = 0

for unique in np.unique(y):
    y = np.where(y==unique,num,y)
    num+=1

In [93]:
y = y.astype(int)

In [94]:
from sklearn.model_selection import train_test_split

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [96]:
del X

In [97]:
del y

In [98]:
del files

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [100]:
early_stop = EarlyStopping(monitor="val_accuracy", mode="max", min_delta=0.005, verbose=7, patience=5)

In [101]:
model = Sequential()

model.add(Conv2D(64, 3, activation='relu', input_shape = (50, 50, 3)))

model.add(MaxPooling2D(2, 2))

model.add(Dropout(0.5))

model.add(Conv2D(32, 3, activation="relu"))

model.add(MaxPooling2D(2 ,2))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation="relu"))

model.add(Dense(9, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [102]:
model.fit(X_train, y_train, epochs=1000, callbacks = [early_stop], validation_data=(X_test,y_test))

In [103]:
model.evaluate(X_test, y_test)[1]*100